
class PtientBasedMBFVariability():
    def __init__(self, args):
        self.args = args

    def ReadPatientsData(self):
        """for each patient two files must exist: Patient_n_FFR_s_MBF_Territories.vtu
        and Patient_n_FFR_s_MBF_Territory_Labels.dat, first containing the MBF values
        and territory assignments, and the latter including the name of all territories.
        """
        pass

    def ExtractIschemicRegions(self, MBF):
        pass

    def CreatePatientsDataFrame(self):
        pass

    def ViolinPlot(self):
        pass

# pipeline:

 - Extract Ischemic/NonIschemic Regions per patients
 - Calculate Flow per regions
 - Create a dataframe containing the following:
    - patient_id
    - Vessel_id
    - territory_id
    - voxel_id
    - Absolute MBF
    - Index MBF
    - Voxel Flow
    - Voxel relative Flow (from Index MBF)
    - total territory flow
    - FFR
    - Stenosis Severity
    - is_stenosed (bool)
 - Create a violin plot of the MBF distribution for severity of 3 and 4
 - Create a histogram plot of the flow based on the severity of the stenosis

In [ ]:
import os 
import vtk
import argparse
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from utilities import ReadVTUFile
from vtk.util.numpy_support import vtk_to_numpy

In [50]:
def ThresholdInBetween(Volume,arrayname,value1,value2):
    Threshold=vtk.vtkThreshold()
    Threshold.SetInputData(Volume)
    #Threshold.ThresholdBetween(value1,value2)
    Threshold.SetLowerThreshold(value1)
    Threshold.SetUpperThreshold(value2)
    #Threshold.SetThresholdFunction(ThresholdInBetween)
    Threshold.SetInputArrayToProcess(0,0,0,vtk.vtkDataObject.FIELD_ASSOCIATION_CELLS,arrayname)
    Threshold.Update()
    return Threshold.GetOutput()

In [26]:
def ReadIschemicLabels(InputLabels):
    Ischemic_Labels = {"post_LAD": [], "post_LCx":[], "post_PL":[], "NonIschemic": []}
    keys = list(Ischemic_Labels.keys())[:-1]
    with open(InputLabels, "r") as ifile:
        for i, LINE in enumerate(ifile):
            if i == 0: continue
            line = LINE.split()
            if line[1].find(keys[0])>=0: Ischemic_Labels[keys[0]].append(int(line[0]))
            elif line[1].find(keys[1])>=0: Ischemic_Labels[keys[1]].append(int(line[0]))
            elif line[1].find(keys[2])>=0: Ischemic_Labels[keys[2]].append(int(line[0]))
            else: Ischemic_Labels["NonIschemic"].append(int(line[0]))


    Ischemic_Labels = {k:v for k, v in Ischemic_Labels.items() if len(v)>0}
    
    return Ischemic_Labels


In [ ]:
from NormalizeMBFMap import MBFNormalization

args = argparse.Namespace()
args.InputFolder = ""
args.InputMBFMap = ""
args.InputLabel = ""
args.ArrayName = "ImageScalars"
args.TerritoryTag = ""

MBFNormalization(args).Normalize()

In [ ]:
Unit = "mm"
def CalculateCellDataFlow(Territory, ArrayName):
    CellData = Territory.GetCellData()
    ImageScalars = vtk_to_numpy(CellData.GetArray(ArrayName))
    NCells = Territory.GetNumberOfCells()
    TerritoryVolume = 0
    TerritoryFlow = []
    for i in range(NCells):
        cell = Territory.GetCell(i)
        cell_bounds = cell.GetBounds()
        cell_volume = abs(cell_bounds[0] - cell_bounds[1]) * abs(cell_bounds[2] - cell_bounds[3]) * abs(cell_bounds[4] - cell_bounds[5])
        TerritoryFlow.append(ImageScalars[i]*cell_volume)
        TerritoryVolume += cell_volume

    if Unit == 'mm':
        return np.array(TerritoryFlow)/1000/100, TerritoryVolume/1000, NCells
    elif Unit == 'cm':
        return np.array(TerritoryFlow)/100, TerritoryVolume, NCells



In [41]:
def ConvertPointDataToCellData(pointdata):
    PointToCell = vtk.vtkPointDataToCellData()
    PointToCell.SetInputData(pointdata)
    PointToCell.Update()

    return PointToCell.GetOutput()

In [52]:
testMBF = "/Users/ana/Documents/AnahitaSeresti/03_MBFValidation/06.MBF_Analysis_Level3/Patient_03/MBF_Territories.vtu"
testLabels = "/Users/ana/Documents/AnahitaSeresti/03_MBFValidation/06.MBF_Analysis_Level3/Patient_03/MBF_Territories_Labels.dat"

MBFMap = ReadVTUFile(testMBF)
#print(MBFMap.GetPointData().GetArrayName(1))
#print(vtk_to_numpy(MBFMap.GetPointData().GetArray("ImageScalars")))
MBFMap_CellData = ConvertPointDataToCellData(MBFMap)
print(vtk_to_numpy(MBFMap_CellData.GetCellData().GetArray(0)))
territory_ = ThresholdInBetween(MBFMap_CellData, "TerritoryMaps", 1, 1)
#territory_cellData = ConvertPointDataToCellData(territory_)
print(territory_.GetCellData().GetArrayName(0))
print(vtk_to_numpy(territory_.GetCellData().GetArray(0)))


Ischemic_Labels = ReadIschemicLabels(testLabels)
Ischemic_Labels


[153.25  153.875 154.5   ...  92.25   91.5    91.   ]
ImageScalars
[153.25  153.875 154.5   ...  64.25   68.25   64.375]


{'post_LAD': [1, 2, 3, 4],
 'post_LCx': [8],
 'post_PL': [9, 10, 11],
 'NonIschemic': [0, 5, 6, 7]}

In [57]:

Flow = {k:0 for k in Ischemic_Labels.keys()}
for (key, value) in Ischemic_Labels.items():
    flow_territory = 0
    for t_id in value:
        territory_ = ThresholdInBetween(MBFMap_CellData, "TerritoryMaps", t_id, t_id)
        flow_, volume, _ = CalculateCellDataFlow(territory_, "ImageScalars")
        print(volume)
        flow_territory += np.sum(flow_)
    Flow[key] = flow_territory

Flow

45.8457233534989
0.25220065099635397
16.74818253580926
30.917719129647192
19.38091500423322
18.387819399243337
21.74493810386816
7.977373578433413
4.708532559078902
12.84626652311941
1.4755917443444377
4.752353259132402


{'post_LAD': 80.11091400382477,
 'post_LCx': 22.190335897350526,
 'post_PL': 52.36001754071448,
 'NonIschemic': 21.372264856816773}

In [ ]:
import glob
def ReadPatientFiles(InputFolder):
    InputFileNames = glob.glob(InputFolder)